## Analysis of RECS data and derive regression models (XGBoost)

Preparation of data
1. Obtain and Filter the raw data to a dataset that consists of key features derived through Feature Engineering 
2. Replace values like 99, 9, -2 with Mode of the column
3. Perform one hot encoding for all categorical values

Running data Models
1. Split train and test data (75:25 ratio)
2. Run XGBOOST algo against train data
3. Use Test to validate prediction
4. Plot the residual and accuracy if possible


In [229]:
# IMport dependencies
import pandas as pd
from pandas import set_option
import numpy
import os
import csv

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

# import required libraries related to xgBoost and sci-kit learn
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# from keras.utils import to_categorical

# import preprocessing from sklearn
from sklearn import preprocessing

# DictVectorizer
from sklearn.feature_extraction import DictVectorizer

### Import Data

# set global variables
dataFilePath = "dataforfinalproject"
filename = "RECS_COMBINED_DATA.csv"
cols_file = "Final_Columns_withCat.csv"

# read dataset wih all years combined data
df_recs = pd.read_csv(os.path.join(dataFilePath, filename), low_memory= False)


# read the columns from Columns csv
df_cols = pd.read_csv(os.path.join(dataFilePath, cols_file))
df_cols.columns

# Whittle down the dataset to contain only Features required for modeling - X 
modelDF = df_recs[df_cols[df_cols.FEATURES_MODEL == "Y"].COLUMN_NAME]
print(f" X Features shape : {modelDF.shape}")

y_label = df_recs['TOTALBTU']
print(f"y label shape : {y_label.shape}")

### Prepare Data

In [235]:
# describe the dataframe that will be used for model
descrDF = modelDF[df_cols[(df_cols.FEATURES_MODEL == "Y")].COLUMN_NAME].describe()
# transpose to make it easier to obtain columns with values of 99 and <0
descrDF = descrDF.transpose().reset_index()
descrDF
# obtain column names with values 99. 99 indicates missing or unavailable info. this needs to be replaced with MOde
cols99_2 = descrDF[(descrDF['max'] == 99.0) | (descrDF['min'] < 0) ]['index'].tolist()
# cols_2 = descrDF[descrDF['min'] == -2.0]['index'].tolist()


In [236]:
print(f"cols with values as 99 and -2: {cols99_2} \n")

# print(f"cols with values as -2 : {cols_2} ")


cols with values as 99 and -2: ['CELLAR', 'STORIES', 'PRKGPLC1', 'BEDROOMS', 'FUELPOOL', 'SIZRFRI1', 'TYPERFR1', 'AGERFRI1', 'SIZRFRI2', 'TYPERFR2', 'AGERFRI2', 'NUMFREEZ', 'SIZFREEZ', 'AGEFRZR', 'STOVENFUEL', 'OVENUSE', 'AMTMICRO', 'DWASHUSE', 'WASHLOAD', 'WASHTEMP', 'EQUIPM', 'FUELHEAT', 'EQUIPAGE', 'PROTHERM', 'TEMPHOME', 'TEMPGONE', 'TEMPNITE', 'AGECENAC', 'USECENAC', 'NUMBERAC', 'WWACAGE', 'USEWWAC', 'FUELH2O', 'WHEATAGE', 'WHEATSIZ', 'LGTOUTNUM', 'ESDISHW', 'PGASHEAT', 'WOODAMT'] 



In [237]:
# For all categorical columns, that have 99 and -2 , replace with Columns Mode value#
# step 1 - Fill na for thse values of 99 and -2
# Step 2: Fillna with mode

# step1 
modelCopy = modelDF.copy()
modelDF[cols99_2] = modelDF[cols99_2].applymap(lambda r : None if r in [99,-2] else r)


#step2 :
# modelDF[cols99_2].apply(lambda r : modes[r.name])
modelDF[cols99_2] = modelDF[cols99_2].fillna(modelDF.mode().iloc[0])

# just for Col EDishw, the values are in -ve  (-9, -8 )so replace it in a separate line
modelDF['ESDISHW'] = modelDF['ESDISHW'].apply(lambda r : 0 if (r < 0) else r)

# check if NAN exists
modelDF.isnull().values.sum()


0

In [238]:
modelDF[df_cols[(df_cols.FEATURES_MODEL == "Y") & (df_cols.COLUMN_TYPE == "Categorical")].COLUMN_NAME].describe()

,REGIONC,DIVISION,CELLAR,PRKGPLC1,YEARMADE,OCCUPYY,WINDOWS,ADQINSUL,DRAFTY,FUELPOOL,...,WHEATSIZ,LGTOUTNUM,ESDISHW,PGASHEAT,HHSEX,EMPLOYHH,HOUSEHOLDER_RACE,MONEYPY,RECBATH,LPWATER
count,26973.000000,26973.000000,26973.000000,26973.000000,26973.000000,26973.000000,26973.000000,26973.000000,26973.000000,26973.000000,...,26973.000000,26973.000000,26973.000000,26973.000000,26973.000000,26973.000000,26973.000000,26973.000000,26973.000000,26973.000000
mean,2.639528,5.323842,1.589071,1.257220,4.219849,6.846587,30.275164,1.861046,3.371297,1.034553,...,2.351018,0.530271,1.355911,2.300597,1.450710,0.837356,1.956475,9.597635,0.376970,0.037334
std,1.051262,2.795930,3.087190,2.298351,2.397493,2.193998,16.054884,0.800860,0.906754,0.429598,...,3.804938,0.994582,2.947997,2.888062,0.497574,0.716003,4.427966,6.728926,1.668587,0.189582
min,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,3.000000,0.000000,0.000000,2.000000,6.000000,20.000000,1.000000,3.000000,1.000000,...,2.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,5.000000,0.000000,0.000000
50%,3.000000,5.000000,0.000000,1.000000,4.000000,8.000000,30.000000,2.000000,4.000000,1.000000,...,2.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,8.000000,0.000000,0.000000
75%,3.000000,7.000000,1.000000,1.000000,6.000000,8.000000,41.000000,2.000000,4.000000,1.000000,...,3.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,13.000000,0.000000,0.000000
max,4.000000,10.000000,9.000000,9.000000,13.000000,9.000000,60.000000,6.000000,6.000000,21.000000,...,95.000000,15.000000,9.000000,9.000000,2.000000,8.000000,42.000000,24.000000,9.000000,1.000000


In [239]:
descrDF1 = modelDF[df_cols[(df_cols.FEATURES_MODEL == "Y")].COLUMN_NAME].describe()
# transpose to make it easier to obtain columns with values of 99 and -2
descrDF1 = descrDF1.transpose().reset_index()
descrDF1[descrDF1['min'] < 0]

,index,count,mean,std,min,25%,50%,75%,max


# XGBOOST - Predicting Total Consumption in BTU

In [240]:
# Drop Price / Cost related Columns as it is only Consumption we are interested in 
cost_cols = df_cols[(df_cols['COLUMN_NAME'].str.find("DOL") != -1) & (df_cols.FEATURES_MODEL == "Y")].COLUMN_NAME.tolist()
cost_cols

['DOLLAREL',
 'DOLELSPH',
 'DOLELCOL',
 'DOLELWTH',
 'DOLELRFG',
 'DOLELOTH',
 'TOTALDOLLAR']

In [241]:
modelDF[cost_cols].head()

,DOLLAREL,DOLELSPH,DOLELCOL,DOLELWTH,DOLELRFG,DOLELOTH,TOTALDOLLAR
0,1667.0,0.0,241.0,376.0,183.0,1105.0,3005.0
1,973.0,0.0,345.0,0.0,70.0,693.0,1744.0
2,820.0,0.0,0.0,267.0,150.0,403.0,820.0
3,4470.0,60.0,698.0,0.0,703.0,3766.0,14198.0
4,1170.0,0.0,113.0,243.0,202.0,751.0,1309.0


In [242]:
modelDF_BTU = modelDF.copy()
modelDF_BTU.drop(cost_cols, axis = 1, inplace = True)
print(modelDF_BTU.shape)
modelDF_BTU.columns

(26973, 80)


Index(['REGIONC', 'DIVISION', 'CELLAR', 'STORIES', 'PRKGPLC1', 'YEARMADE',
       'OCCUPYY', 'BEDROOMS', 'NCOMBATH', 'NHAFBATH', 'OTHROOMS', 'TOTROOMS',
       'WINDOWS', 'ADQINSUL', 'DRAFTY', 'FUELPOOL', 'SIZRFRI1', 'TYPERFR1',
       'AGERFRI1', 'SIZRFRI2', 'TYPERFR2', 'AGERFRI2', 'NUMFREEZ', 'SIZFREEZ',
       'AGEFRZR', 'STOVENFUEL', 'OVENUSE', 'AMTMICRO', 'NUMMEAL', 'DWASHUSE',
       'WASHLOAD', 'WASHTEMP', 'EQUIPM', 'FUELHEAT', 'EQUIPAGE', 'PROTHERM',
       'TEMPHOME', 'TEMPGONE', 'TEMPNITE', 'ELECAUX', 'AGECENAC', 'USECENAC',
       'NUMBERAC', 'WWACAGE', 'USEWWAC', 'NUMCFAN', 'FUELH2O', 'WHEATAGE',
       'WHEATSIZ', 'LGTOUTNUM', 'ESDISHW', 'PGASHEAT', 'HHSEX', 'HHAGE',
       'EMPLOYHH', 'HOUSEHOLDER_RACE', 'NHSLDMEM', 'MONEYPY', 'TOTCSQFT',
       'TOTHSQFT', 'TOTUSQFT', 'CDD65', 'HDD65', 'GALLONLP', 'CUFEETNG',
       'BTUEL', 'BTUELSPH', 'BTUELCOL', 'BTUELWTH', 'BTUELRFG', 'BTUELOTH',
       'BTUFO', 'TOTALBTU', 'WOODAMT', 'RECBATH', 'NUMFRIG', 'LPWATER',
       'TVAUDIOE

In [243]:
# assign target or output to y
y = modelDF_BTU['TOTALBTU']
print(f"shape of y is {y.shape}")

# and drop TOTAL BTU from X set
X = modelDF_BTU.drop(['TOTALBTU'], axis = 1)
print(f"shape of X is {X.shape}")

shape of y is (26973,)
shape of X is (26973, 79)


### Apply dict vectorizer 

In [244]:
X_dict = X.to_dict(orient = "records")
X_dict

[{'REGIONC': 1.0,
  'DIVISION': 1.0,
  'CELLAR': 1.0,
  'STORIES': 31.0,
  'PRKGPLC1': 1.0,
  'YEARMADE': 9.0,
  'OCCUPYY': 9.0,
  'BEDROOMS': 6.0,
  'NCOMBATH': 2.0,
  'NHAFBATH': 1.0,
  'OTHROOMS': 4.0,
  'TOTROOMS': 13.0,
  'WINDOWS': 6.0,
  'ADQINSUL': 2.0,
  'DRAFTY': 3.0,
  'FUELPOOL': 1.0,
  'SIZRFRI1': 4.0,
  'TYPERFR1': 2.0,
  'AGERFRI1': 2.0,
  'SIZRFRI2': 9.0,
  'TYPERFR2': 9.0,
  'AGERFRI2': 3.0,
  'NUMFREEZ': 0.0,
  'SIZFREEZ': 9.0,
  'AGEFRZR': 9.0,
  'STOVENFUEL': 9.0,
  'OVENUSE': 9.0,
  'AMTMICRO': 2.0,
  'NUMMEAL': 4.0,
  'DWASHUSE': 13.0,
  'WASHLOAD': 3.0,
  'WASHTEMP': 3.0,
  'EQUIPM': 3.0,
  'FUELHEAT': 1.0,
  'EQUIPAGE': 3.0,
  'PROTHERM': 0.0,
  'TEMPHOME': 70.0,
  'TEMPGONE': 65.0,
  'TEMPNITE': 70.0,
  'ELECAUX': 0.0,
  'AGECENAC': 3.0,
  'USECENAC': 2.0,
  'NUMBERAC': 0.0,
  'WWACAGE': 9.0,
  'USEWWAC': 9.0,
  'NUMCFAN': 0.0,
  'FUELH2O': 5.0,
  'WHEATAGE': 3.0,
  'WHEATSIZ': 3.0,
  'LGTOUTNUM': 0.0,
  'ESDISHW': 0.0,
  'PGASHEAT': 1.0,
  'HHSEX': 1.0,
  'HHA

In [246]:
# instantiate a Dictvectorizer object for X
dv_X = DictVectorizer(sparse=False) 
# sparse = False makes the output is not a sparse matrix

In [247]:
# apply dv_X on X_dict
X_encoded = dv_X.fit_transform(X_dict)
# show X_encoded
X_encoded

array([[ 2.,  3.,  9., ...,  0.,  9.,  9.],
       [ 2.,  1.,  9., ...,  0.,  9.,  7.],
       [ 2.,  3.,  9., ...,  0.,  9.,  4.],
       ...,
       [ 3.,  3.,  1., ...,  0.,  1.,  7.],
       [ 1., 41.,  9., ...,  0.,  9.,  7.],
       [ 2.,  3.,  9., ...,  0.,  9.,  4.]])

In [248]:
# vocabulary
vocab = dv_X.vocabulary_
# show vocab
vocab

{'REGIONC': 53,
 'DIVISION': 17,
 'CELLAR': 15,
 'STORIES': 57,
 'PRKGPLC1': 50,
 'YEARMADE': 78,
 'OCCUPYY': 44,
 'BEDROOMS': 6,
 'NCOMBATH': 36,
 'NHAFBATH': 37,
 'OTHROOMS': 45,
 'TOTROOMS': 64,
 'WINDOWS': 75,
 'ADQINSUL': 0,
 'DRAFTY': 18,
 'FUELPOOL': 27,
 'SIZRFRI1': 55,
 'TYPERFR1': 67,
 'AGERFRI1': 3,
 'SIZRFRI2': 56,
 'TYPERFR2': 68,
 'AGERFRI2': 4,
 'NUMFREEZ': 41,
 'SIZFREEZ': 54,
 'AGEFRZR': 2,
 'STOVENFUEL': 58,
 'OVENUSE': 46,
 'AMTMICRO': 5,
 'NUMMEAL': 43,
 'DWASHUSE': 19,
 'WASHLOAD': 71,
 'WASHTEMP': 72,
 'EQUIPM': 23,
 'FUELHEAT': 26,
 'EQUIPAGE': 22,
 'PROTHERM': 51,
 'TEMPHOME': 60,
 'TEMPGONE': 59,
 'TEMPNITE': 61,
 'ELECAUX': 20,
 'AGECENAC': 1,
 'USECENAC': 69,
 'NUMBERAC': 39,
 'WWACAGE': 77,
 'USEWWAC': 70,
 'NUMCFAN': 40,
 'FUELH2O': 25,
 'WHEATAGE': 73,
 'WHEATSIZ': 74,
 'LGTOUTNUM': 33,
 'ESDISHW': 24,
 'PGASHEAT': 48,
 'HHSEX': 31,
 'HHAGE': 30,
 'EMPLOYHH': 21,
 'HOUSEHOLDER_RACE': 32,
 'NHSLDMEM': 38,
 'MONEYPY': 35,
 'TOTCSQFT': 62,
 'TOTHSQFT': 63,
 '

In [249]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [250]:
X_train.shape

(21578, 79)

### Apply one hot encoding on the dataset for all categorical columns

In [ ]:
# TODO: create a OneHotEncoder object, and fit it to all of Categorical Values of X (independent variable)

# dataset that needs OHE
XY = modelDF[df_cols[(df_cols.FEATURES_MODEL == "Y")].COLUMN_NAME]
print(XY.shape)

In [180]:
# CreateCategory Column mask to identify the columns that are categorical 

CATCOLS = df_cols[(df_cols.FEATURES_MODEL == "Y") & (df_cols.COLUMN_TYPE == "Categorical")].COLUMN_NAME.tolist()
XY_CAT_COLS_MASK = modelDF.columns.isin(CATCOLS)

print(XY_CAT_COLS_MASK)
print(len(XY_CAT_COLS_MASK))


[ True  True  True False  True  True  True False False False False False
  True  True  True  True  True  True  True  True  True  True False  True
  True  True False False  True False False  True  True  True  True  True
 False False False  True  True  True False  True  True False  True  True
  True  True  True  True  True False  True  True False  True False False
 False False False False False False False False False False False False
 False False False False False False False False  True False  True False
 False False]
86


In [182]:
# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder(categorical_features = XY_CAT_COLS_MASK, sparse= False)

# 2. FIT
enc.fit(XY)

OneHotEncoder(categorical_features=array([ True,  True, ..., False, False]),
       dtype=<class 'numpy.float64'>, handle_unknown='error',
       n_values='auto', sparse=False)

In [184]:
# 3. Transform
onehotlabels = enc.transform(XY)
onehotlabels.shape

# as you can see, you've the same number of rows 891
# but now you've so many more columns due to how we changed all the categorical data into numerical data

(26973, 372)

In [185]:
onehotlabels

array([[1., 0., 0., ..., 8., 2., 3.],
       [0., 0., 1., ..., 7., 2., 3.],
       [0., 0., 0., ..., 9., 0., 3.],
       ...,
       [0., 1., 0., ..., 4., 0., 1.],
       [0., 0., 1., ..., 4., 2., 2.],
       [0., 1., 0., ..., 6., 2., 2.]])